This notebook explores both model index and opc ua scripts and contain examples of all the functions to make request to model index api and opc ua api servers.  

### Import Libraries

In [1]:
%load_ext autotime

time: 1.24 ms (started: 2022-08-26 10:57:41 +02:00)


In [2]:
# Import the required packeages
import pandas as pd
import urllib3
urllib3.disable_warnings()
from pandas.io.json import json_normalize
from typing import List, Dict
import sys
import os
from aiohttp import ClientSession
import asyncio
import dask.dataframe as dd

time: 1.43 s (started: 2022-08-26 10:57:41 +02:00)


### Import Scripts

In [3]:
# Setting the path
module_path = os.path.abspath(os.path.join("../src/pyprediktormapclient/"))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/home/meena/dev/migrate/pyPrediktorMapClient/src/pyprediktormapclient
time: 1.1 ms (started: 2022-08-26 10:57:43 +02:00)


In [4]:
# Import model index functions
from model_index import ModelIndex

# Import OPC UA functions
from opc_ua import OPC_UA

time: 1.17 s (started: 2022-08-26 10:57:43 +02:00)


In [5]:
# Connection to the servers
model_index_url = "http://10.241.68.86:7001/v1/"
opc_url = "http://10.241.68.86:13371/"

# Model index API
mdx = ModelIndex(url=model_index_url)

# OPC UA API
opc = OPC_UA(url=opc_url)

time: 61.1 ms (started: 2022-08-26 10:57:44 +02:00)


In [6]:
# Input parameters for value data
server_url = "opc.tcp://10.241.80.4:4872"
# Parameters for aggregate historical data
start_time = "2022-07-05T07:55:14.544000Z" # 5th July aggregated data 
end_time = "2022-07-06T07:55:14.544000Z"
pro_interval = 600000 # 10 minutes processing time
agg_name = "Average"

time: 549 µs (started: 2022-08-26 10:57:44 +02:00)


### Download data from modelindex api

In [7]:
# Listed sites on the model index api server
namespaces = mdx.get_namespace_array()
namespaces

,Idx,Uri
0,0,http://opcfoundation.org/UA/
1,1,http://prediktor.no/apis/ua/
2,2,urn:prediktor:NF-SXD9E-021.i04.local:ScatecDES
3,3,http://scatecsolar.com/BR-AP
4,15,http://prediktor.no/PVTypes/
5,19,http://scatecsolar.com/Enterprise
6,4,http://scatecsolar.com/EG-AS
7,5,http://scatecsolar.com/EG-DA
8,6,http://scatecsolar.com/EG-KO
9,7,http://scatecsolar.com/EG-RS


time: 45.4 ms (started: 2022-08-26 10:57:44 +02:00)


In [8]:
# Types of Objects
obj_types = pd.DataFrame(mdx.get_object_types())
obj_types

,Id,DisplayName,BrowseName,Props,Vars
0,15:0:1004,SignalSetType,SignalSetType,[],[]
1,15:0:1002,PVAssetType,PVAssetType,[],[]
2,15:0:1003,ParameterSetType,ParameterSetType,[],[]
3,15:0:1004,SignalSetType,SignalSetType,[],[]
4,15:0:1005,MethodSetType,MethodSetType,[],[]
...,...,...,...,...,...
967,15:0:1024,GridConnectionPointType,GridConnectionPointType,[],[]
968,15:0:1025,EnterpriseType,EnterpriseType,[],[]
969,15:0:1026,ObjectStatusType,ObjectStatusType,[],[]
970,15:0:1027,ModuleMountingStructureType,ModuleMountingStructureType,[],[]


time: 78.5 ms (started: 2022-08-26 10:57:45 +02:00)


In [9]:
# Unique types of Objects
obj_types = pd.DataFrame(mdx.get_object_types())
obj_types_unique = obj_types[['Id', 'DisplayName', 'BrowseName']].drop_duplicates()
obj_types_unique

,Id,DisplayName,BrowseName
0,15:0:1004,SignalSetType,SignalSetType
1,15:0:1002,PVAssetType,PVAssetType
2,15:0:1003,ParameterSetType,ParameterSetType
4,15:0:1005,MethodSetType,MethodSetType
5,15:0:1006,AlarmSetType,AlarmSetType
6,15:0:1007,PVUnitType,PVUnitType
7,15:0:1008,PVEquipmentType,PVEquipmentType
8,15:0:1009,SiteType,SiteType
9,15:0:1010,SubSiteType,SubSiteType
10,15:0:1011,SectionType,SectionType


time: 88.8 ms (started: 2022-08-26 10:57:45 +02:00)


In [10]:
# To get typeId by type name of an object
object_type_id = mdx.get_object_typeid("SiteType")
object_type_id

'15:0:1009'

time: 6.55 ms (started: 2022-08-26 10:57:46 +02:00)


In [11]:
# To get the objects of a type
obj_of_types = mdx.get_objects_of_type("SiteType")
obj_of_types

,Id,type,subtype,DisplayName,Props,Vars
0,3:1:SSO.BR-AP,15:0:1009,15:0:1009,BR-AP,"[{'DisplayName': 'CommercialOperationDate', 'V...","[{'DisplayName': 'Altitude', 'Id': '3:1:SSO.BR..."
1,4:1:SSO.EG-AS,15:0:1009,15:0:1009,EG-AS,"[{'DisplayName': 'CommercialOperationDate', 'V...","[{'DisplayName': 'CPR', 'Id': '4:1:SSO.EG-AS.M..."
2,5:1:SSO.EG-DA,15:0:1009,15:0:1009,EG-DA,"[{'DisplayName': 'CommercialOperationDate', 'V...","[{'DisplayName': 'CPR', 'Id': '5:1:SSO.EG-DA.M..."
3,6:1:SSO.EG-KO,15:0:1009,15:0:1009,EG-KO,"[{'DisplayName': 'CommercialOperationDate', 'V...","[{'DisplayName': 'CPR', 'Id': '6:1:SSO.EG-KO.M..."
4,7:1:SSO.EG-RS,15:0:1009,15:0:1009,EG-RS,"[{'DisplayName': 'CommercialOperationDate', 'V...","[{'DisplayName': 'CPR', 'Id': '7:1:SSO.EG-RS.M..."
5,8:1:SSO.EG-UE,15:0:1009,15:0:1009,EG-UE,"[{'DisplayName': 'CommercialOperationDate', 'V...","[{'DisplayName': 'CPR', 'Id': '8:1:SSO.EG-UE.M..."
6,9:1:SSO.EG-ZA,15:0:1009,15:0:1009,EG-ZA,"[{'DisplayName': 'CommercialOperationDate', 'V...","[{'DisplayName': 'CPR', 'Id': '9:1:SSO.EG-ZA.M..."
7,10:1:SSO.HN-AF,15:0:1009,15:0:1009,HN-AF,"[{'DisplayName': 'CommercialOperationDate', 'V...","[{'DisplayName': 'CPR', 'Id': '10:1:SSO.HN-AF...."
8,11:1:SSO.HN-LP,15:0:1009,15:0:1009,HN-LP,"[{'DisplayName': 'CommercialOperationDate', 'V...","[{'DisplayName': 'Altitude', 'Id': '11:1:SSO.H..."
9,12:1:SSO.JO-EJ,15:0:1009,15:0:1009,JO-EJ,"[{'DisplayName': 'CommercialOperationDate', 'V...","[{'DisplayName': 'CPR', 'Id': '12:1:SSO.JO-EJ...."


time: 229 ms (started: 2022-08-26 10:57:50 +02:00)


In [13]:
# Descendents of an object type
obj_descendents = mdx.get_object_descendants("StringSetType", obj_of_types, "PV_Assets")
obj_descendents

,ObjectId,DescendantId,DescendantName,DescendantType,ObjectName,Vars,Props
0,3:1:SSO.BR-AP,3:1:SSO.BR-AP.AP1.Z1.TS11.I11.01.SM01.CH01,BR-AP-TS11.I01-SM01-CH01,StringSetType,BR-AP,"[{'DisplayName': 'Height', 'Id': '3:1:SSO.BR-A...","[{'DisplayName': 'ColPosition', 'Value': '1'},..."
1,3:1:SSO.BR-AP,3:1:SSO.BR-AP.AP1.Z1.TS11.I11.01.SM01.CH02,BR-AP-TS11.I01-SM01-CH02,StringSetType,BR-AP,"[{'DisplayName': 'Height', 'Id': '3:1:SSO.BR-A...","[{'DisplayName': 'ColPosition', 'Value': '2'},..."
2,3:1:SSO.BR-AP,3:1:SSO.BR-AP.AP1.Z1.TS11.I11.01.SM01.CH03,BR-AP-TS11.I01-SM01-CH03,StringSetType,BR-AP,"[{'DisplayName': 'Height', 'Id': '3:1:SSO.BR-A...","[{'DisplayName': 'ColPosition', 'Value': '3'},..."
3,3:1:SSO.BR-AP,3:1:SSO.BR-AP.AP1.Z1.TS11.I11.01.SM01.CH04,BR-AP-TS11.I01-SM01-CH04,StringSetType,BR-AP,"[{'DisplayName': 'Height', 'Id': '3:1:SSO.BR-A...","[{'DisplayName': 'ColPosition', 'Value': '4'},..."
4,3:1:SSO.BR-AP,3:1:SSO.BR-AP.AP1.Z1.TS11.I11.01.SM01.CH06,BR-AP-TS11.I01-SM01-CH06,StringSetType,BR-AP,"[{'DisplayName': 'Height', 'Id': '3:1:SSO.BR-A...","[{'DisplayName': 'ColPosition', 'Value': '6'},..."
...,...,...,...,...,...,...,...
119853,40:1:SSO.AR-GU,40:1:SSO.AR-GU.S1.Z5.TS05.I05.I21.1.SMI21.1.CH04,AR-GU-TS05.I21.1-SMI21.1-CH04,StringSetType,AR-GU,"[{'DisplayName': 'Height', 'Id': '40:1:SSO.AR-...","[{'DisplayName': 'ColPosition', 'Value': '3'},..."
119854,40:1:SSO.AR-GU,40:1:SSO.AR-GU.S1.Z5.TS05.I05.I21.1.SMI21.1.CH06,AR-GU-TS05.I21.1-SMI21.1-CH06,StringSetType,AR-GU,"[{'DisplayName': 'Height', 'Id': '40:1:SSO.AR-...","[{'DisplayName': 'ColPosition', 'Value': '3'},..."
119855,40:1:SSO.AR-GU,40:1:SSO.AR-GU.S1.Z5.TS05.I05.I21.1.SMI21.1.CH07,AR-GU-TS05.I21.1-SMI21.1-CH07,StringSetType,AR-GU,"[{'DisplayName': 'Height', 'Id': '40:1:SSO.AR-...","[{'DisplayName': 'ColPosition', 'Value': '3'},..."
119856,40:1:SSO.AR-GU,40:1:SSO.AR-GU.S1.Z5.TS05.I05.I21.1.SMI21.1.CH11,AR-GU-TS05.I21.1-SMI21.1-CH11,StringSetType,AR-GU,"[{'DisplayName': 'Height', 'Id': '40:1:SSO.AR-...","[{'DisplayName': 'ColPosition', 'Value': '4'},..."


time: 20 s (started: 2022-08-26 10:59:06 +02:00)


In [14]:
# All the sites on the OPC server
sites = mdx.get_objects_of_type('SiteType')

# Selecting specific site 
sitename = 'BR-AP'
site = sites[sites['DisplayName'] == sitename]

time: 54.8 ms (started: 2022-08-26 10:59:26 +02:00)


In [15]:
# Object descendants data of a specific site BR-AP
strings = mdx.get_object_descendants("StringSetType", site, "PV_Assets")
strings

,ObjectId,DescendantId,DescendantName,DescendantType,ObjectName,Vars,Props
0,3:1:SSO.BR-AP,3:1:SSO.BR-AP.AP1.Z1.TS11.I11.01.SM01.CH01,BR-AP-TS11.I01-SM01-CH01,StringSetType,BR-AP,"[{'DisplayName': 'Height', 'Id': '3:1:SSO.BR-A...","[{'DisplayName': 'ColPosition', 'Value': '1'},..."
1,3:1:SSO.BR-AP,3:1:SSO.BR-AP.AP1.Z1.TS11.I11.01.SM01.CH02,BR-AP-TS11.I01-SM01-CH02,StringSetType,BR-AP,"[{'DisplayName': 'Height', 'Id': '3:1:SSO.BR-A...","[{'DisplayName': 'ColPosition', 'Value': '2'},..."
2,3:1:SSO.BR-AP,3:1:SSO.BR-AP.AP1.Z1.TS11.I11.01.SM01.CH03,BR-AP-TS11.I01-SM01-CH03,StringSetType,BR-AP,"[{'DisplayName': 'Height', 'Id': '3:1:SSO.BR-A...","[{'DisplayName': 'ColPosition', 'Value': '3'},..."
3,3:1:SSO.BR-AP,3:1:SSO.BR-AP.AP1.Z1.TS11.I11.01.SM01.CH04,BR-AP-TS11.I01-SM01-CH04,StringSetType,BR-AP,"[{'DisplayName': 'Height', 'Id': '3:1:SSO.BR-A...","[{'DisplayName': 'ColPosition', 'Value': '4'},..."
4,3:1:SSO.BR-AP,3:1:SSO.BR-AP.AP1.Z1.TS11.I11.01.SM01.CH05,BR-AP-TS11.I01-SM01-CH05,StringSetType,BR-AP,"[{'DisplayName': 'Height', 'Id': '3:1:SSO.BR-A...","[{'DisplayName': 'ColPosition', 'Value': '5'},..."
...,...,...,...,...,...,...,...
8083,3:1:SSO.BR-AP,3:1:SSO.BR-AP.AP4.Z8.TS49.I49.04.SM05.CH07,BR-AP-TS49.I04-SM05-CH07,StringSetType,BR-AP,"[{'DisplayName': 'Height', 'Id': '3:1:SSO.BR-A...","[{'DisplayName': 'ColPosition', 'Value': '55'}..."
8084,3:1:SSO.BR-AP,3:1:SSO.BR-AP.AP4.Z8.TS49.I49.04.SM05.CH08,BR-AP-TS49.I04-SM05-CH08,StringSetType,BR-AP,"[{'DisplayName': 'Height', 'Id': '3:1:SSO.BR-A...","[{'DisplayName': 'ColPosition', 'Value': '56'}..."
8085,3:1:SSO.BR-AP,3:1:SSO.BR-AP.AP4.Z8.TS49.I49.04.SM05.CH09,BR-AP-TS49.I04-SM05-CH09,StringSetType,BR-AP,"[{'DisplayName': 'Height', 'Id': '3:1:SSO.BR-A...","[{'DisplayName': 'ColPosition', 'Value': '57'}..."
8086,3:1:SSO.BR-AP,3:1:SSO.BR-AP.AP4.Z8.TS49.I49.04.SM05.CH10,BR-AP-TS49.I04-SM05-CH10,StringSetType,BR-AP,"[{'DisplayName': 'Height', 'Id': '3:1:SSO.BR-A...","[{'DisplayName': 'ColPosition', 'Value': '58'}..."


time: 1.64 s (started: 2022-08-26 10:59:26 +02:00)


In [16]:
# Ancestors of an object type
obj_ancestors = mdx.get_object_ancestors("TrackerType", strings, "PV_Serves")
obj_ancestors 

,ObjectId,AncestorId,AncestorName,AncestorType,ObjectName,Vars,Props
0,3:1:SSO.BR-AP.AP1.Z1.TS11.I11.01.SM01.CH01,3:1:SSO.BR-AP.AP1.M1.TR.T11A09,BR-AP-TR.T11A09,TrackerType,BR-AP-TS11.I01-SM01-CH01,"[{'DisplayName': 'TrackingLimitEastAngle', 'Id...","[{'DisplayName': 'Category', 'Value': 'Single ..."
1,3:1:SSO.BR-AP.AP1.Z1.TS11.I11.01.SM01.CH02,3:1:SSO.BR-AP.AP1.M1.TR.T11B09,BR-AP-TR.T11B09,TrackerType,BR-AP-TS11.I01-SM01-CH02,"[{'DisplayName': 'TrackingLimitEastAngle', 'Id...","[{'DisplayName': 'Category', 'Value': 'Single ..."
2,3:1:SSO.BR-AP.AP1.Z1.TS11.I11.01.SM01.CH03,3:1:SSO.BR-AP.AP1.M1.TR.T11B09,BR-AP-TR.T11B09,TrackerType,BR-AP-TS11.I01-SM01-CH03,"[{'DisplayName': 'TrackingLimitEastAngle', 'Id...","[{'DisplayName': 'Category', 'Value': 'Single ..."
3,3:1:SSO.BR-AP.AP1.Z1.TS11.I11.01.SM01.CH04,3:1:SSO.BR-AP.AP1.M1.TR.T11A10,BR-AP-TR.T11A10,TrackerType,BR-AP-TS11.I01-SM01-CH04,"[{'DisplayName': 'TrackingLimitEastAngle', 'Id...","[{'DisplayName': 'Category', 'Value': 'Single ..."
4,3:1:SSO.BR-AP.AP1.Z1.TS11.I11.01.SM01.CH05,3:1:SSO.BR-AP.AP1.M1.TR.T11A10,BR-AP-TR.T11A10,TrackerType,BR-AP-TS11.I01-SM01-CH05,"[{'DisplayName': 'TrackingLimitEastAngle', 'Id...","[{'DisplayName': 'Category', 'Value': 'Single ..."
...,...,...,...,...,...,...,...
8083,3:1:SSO.BR-AP.AP4.Z8.TS49.I49.04.SM05.CH07,3:1:SSO.BR-AP.AP4.M1.TR.T49C11,BR-AP-TR.T49C11,TrackerType,BR-AP-TS49.I04-SM05-CH07,"[{'DisplayName': 'TrackingLimitEastAngle', 'Id...","[{'DisplayName': 'Category', 'Value': 'Single ..."
8084,3:1:SSO.BR-AP.AP4.Z8.TS49.I49.04.SM05.CH08,3:1:SSO.BR-AP.AP4.M1.TR.T49C11,BR-AP-TR.T49C11,TrackerType,BR-AP-TS49.I04-SM05-CH08,"[{'DisplayName': 'TrackingLimitEastAngle', 'Id...","[{'DisplayName': 'Category', 'Value': 'Single ..."
8085,3:1:SSO.BR-AP.AP4.Z8.TS49.I49.04.SM05.CH09,3:1:SSO.BR-AP.AP4.M1.TR.T49D11,BR-AP-TR.T49D11,TrackerType,BR-AP-TS49.I04-SM05-CH09,"[{'DisplayName': 'TrackingLimitEastAngle', 'Id...","[{'DisplayName': 'Category', 'Value': 'Single ..."
8086,3:1:SSO.BR-AP.AP4.Z8.TS49.I49.04.SM05.CH10,3:1:SSO.BR-AP.AP4.M1.TR.T49C12,BR-AP-TR.T49C12,TrackerType,BR-AP-TS49.I04-SM05-CH10,"[{'DisplayName': 'TrackingLimitEastAngle', 'Id...","[{'DisplayName': 'Category', 'Value': 'Single ..."


time: 1.66 s (started: 2022-08-26 10:59:28 +02:00)


### Download data from the opc ua api

In [17]:
# Input parameters for value data
server_url = "opc.tcp://10.241.80.4:4872"
# For aggregate historical data
start_time = "2022-07-05T07:55:14.544000Z" # 5th July aggregated data 
end_time = "2022-07-06T07:55:14.544000Z"
pro_interval = 3600000 # 1 hour processing time
agg_name = "Average"

time: 715 µs (started: 2022-08-26 10:59:30 +02:00)


In [18]:
# Live value data of trackers 
live_value = opc.get_live_values_data(mdx, server_url, ['AngleMeasured', 'AngleSetpoint'], obj_ancestors)
live_value

,Id,Name,Variable,Timestamp,Value
0,3:1:SSO.BR-AP.AP1.M1.TR.T11A09.Signals.AngleMe...,BR-AP-TR.T11A09,AngleMeasured,2022-08-26T08:56:36.8756757Z,-13.5
1,3:1:SSO.BR-AP.AP1.M1.TR.T11A09.Signals.AngleSe...,BR-AP-TR.T11A09,AngleSetpoint,2022-08-26T08:56:36.8756757Z,-14.1
2,3:1:SSO.BR-AP.AP1.M1.TR.T11B09.Signals.AngleMe...,BR-AP-TR.T11B09,AngleMeasured,2022-08-26T08:56:36.8756757Z,-10.7
3,3:1:SSO.BR-AP.AP1.M1.TR.T11B09.Signals.AngleSe...,BR-AP-TR.T11B09,AngleSetpoint,2022-08-26T08:56:36.8756757Z,-11.2
4,3:1:SSO.BR-AP.AP1.M1.TR.T11B09.Signals.AngleMe...,BR-AP-TR.T11B09,AngleMeasured,2022-08-26T08:56:36.8756757Z,-10.7
...,...,...,...,...,...
16171,3:1:SSO.BR-AP.AP4.M1.TR.T49D11.Signals.AngleSe...,BR-AP-TR.T49D11,AngleSetpoint,2022-08-26T08:57:36.8756757Z,-15.7
16172,3:1:SSO.BR-AP.AP4.M1.TR.T49C12.Signals.AngleMe...,BR-AP-TR.T49C12,AngleMeasured,2022-08-26T08:57:36.8756757Z,-13.4
16173,3:1:SSO.BR-AP.AP4.M1.TR.T49C12.Signals.AngleSe...,BR-AP-TR.T49C12,AngleSetpoint,2022-08-26T08:57:36.8756757Z,-14.3
16174,3:1:SSO.BR-AP.AP4.M1.TR.T49C12.Signals.AngleMe...,BR-AP-TR.T49C12,AngleMeasured,2022-08-26T08:57:36.8756757Z,-13.4


time: 24.7 s (started: 2022-08-26 10:59:30 +02:00)


In [19]:
# 1 day aggregated historical trackers data
session = ClientSession()
await opc.get_agg_hist_value_data(session,server_url, start_time, end_time, pro_interval, agg_name, obj_ancestors, ['AngleMeasured', 'AngleSetpoint'])
await session.close()

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f4433dfe730>


time: 2min 55s (started: 2022-08-26 10:59:55 +02:00)


In [20]:
# Reading downloaded tracker data 
tracker_df = pd.read_parquet('Data/data_chunk_0.parquet')
tracker_df

,Id,Variable,Value,Timestamp,Code,Quality
0,SSO.BR-AP.AP1.M1.TR.T11A09.Signals.AngleMeasured,AngleMeasured,-0.621667,2022-07-05T07:55:14.544Z,1,Good
1,SSO.BR-AP.AP1.M1.TR.T11A09.Signals.AngleMeasured,AngleMeasured,-31.808333,2022-07-05T08:55:14.544Z,1,Good
2,SSO.BR-AP.AP1.M1.TR.T11A09.Signals.AngleMeasured,AngleMeasured,-59.561667,2022-07-05T09:55:14.544Z,1,Good
3,SSO.BR-AP.AP1.M1.TR.T11A09.Signals.AngleMeasured,AngleMeasured,-48.708333,2022-07-05T10:55:14.544Z,1,Good
4,SSO.BR-AP.AP1.M1.TR.T11A09.Signals.AngleMeasured,AngleMeasured,-33.243333,2022-07-05T11:55:14.544Z,1,Good
...,...,...,...,...,...,...
388219,SSO.BR-AP.AP4.M1.TR.T49C12.Signals.AngleSetpoint,AngleSetpoint,0.000000,2022-07-06T02:55:14.544Z,1,Good
388220,SSO.BR-AP.AP4.M1.TR.T49C12.Signals.AngleSetpoint,AngleSetpoint,0.000000,2022-07-06T03:55:14.544Z,1,Good
388221,SSO.BR-AP.AP4.M1.TR.T49C12.Signals.AngleSetpoint,AngleSetpoint,0.000000,2022-07-06T04:55:14.544Z,1,Good
388222,SSO.BR-AP.AP4.M1.TR.T49C12.Signals.AngleSetpoint,AngleSetpoint,0.000000,2022-07-06T05:55:14.544Z,1,Good


time: 170 ms (started: 2022-08-26 11:02:51 +02:00)
